In [1]:
import tushare as ts
import pandas as pd

token = '82f17f80a6f62681bcbf105689d892a9559a4e65af4045adb472eaf0'
ts.set_token(token)

In [2]:
start_date = "2000-01-01"
end_date = "2015-01-01"

df1 = ts.pro_bar('600000.SH', adj='qfq',
                 start_date=start_date, end_date=end_date)
df1["trade_date"] = pd.to_datetime(df1["trade_date"])
df1.sort_values(by="trade_date", inplace=True)
df1.reset_index(drop=True, inplace=True)
df1["return_daily"] = df1["pct_chg"] / 100

df1.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,return_daily
0,600000.SH,2000-01-04,3.19,3.30,3.16,3.27,3.16,0.11,3.48,44961.0,113946.783,0.0348
1,600000.SH,2000-01-05,3.27,3.32,3.22,3.23,3.27,-0.04,-1.22,52528.0,134465.427,-0.0122
2,600000.SH,2000-01-06,3.22,3.36,3.20,3.32,3.23,0.09,2.79,62297.0,160059.798,0.0279
3,600000.SH,2000-01-07,3.36,3.52,3.34,3.44,3.32,0.12,3.61,213553.0,575751.082,0.0361
4,600000.SH,2000-01-10,3.45,3.56,3.42,3.48,3.44,0.04,1.16,165397.0,450453.518,0.0116


In [3]:
def get_ewm_vol(df, span=20):
    return df["return_daily"].ewm(span=span).std()


pd.DataFrame(get_ewm_vol(df1))[20:25]

,return_daily
20,0.037921
21,0.036482
22,0.034745
23,0.032912
24,0.034219


In [4]:
events = df1[["trade_date", "close", "return_daily"]].copy(deep=True)
events["end_date"] = events["trade_date"].shift(-20)
events["Vol"] = get_ewm_vol(df1)
events.head()

,trade_date,close,return_daily,end_date,Vol
0,2000-01-04,3.27,0.0348,2000-02-15,NaN
1,2000-01-05,3.23,-0.0122,2000-02-16,0.033234
2,2000-01-06,3.32,0.0279,2000-02-17,0.025202
3,2000-01-07,3.44,0.0361,2000-02-18,0.022492
4,2000-01-10,3.48,0.0116,2000-02-21,0.019649


In [5]:
def get_bound(events, width):
    middle_ = events["return_daily"].ewm(span=20).mean()
    if width[0] > 0:
        events["UpperBound"] = events["Vol"]*width[0]+middle_
    else:
        events["UpperBound"] = np.nan
    if width[1] > 0:
        events["LowerBound"] = -events["Vol"]*width[1]+middle_
    else:
        events["LowerBound"] = np.nan
    return events


def get_bound_time(events):
    result = events.copy(deep=True)
    for i in range(len(events)-20):
        date_cur, date_future = events.loc[i,
                                           "trade_date"], events.loc[i, "end_date"]
        df_date_range = events[(events["trade_date"] > date_cur) & (
            events["trade_date"] < date_future)].copy(deep=True)
        close_first = df_date_range.iloc[0]["close"]
        df_date_range["CumReturn"] = df_date_range["close"]/close_first-1
        result.loc[i, "UpperTime"] = df_date_range.loc[df_date_range["CumReturn"]
                                                       > df_date_range["UpperBound"].iloc[0], "trade_date"].min()
        result.loc[i, "LowerTime"] = df_date_range.loc[df_date_range["CumReturn"]
                                                       < df_date_range["LowerBound"].iloc[0], "trade_date"].min()
        result.loc[i, "FirstTime"] = min(result.loc[i, "end_date"], result.loc[i, "UpperTime"],
                                         result.loc[i, "LowerTime"])

    return result


events = get_bound(events, [2, 2])
events = get_bound_time(events)[20:].reset_index(inplace=False, drop=True)
events[20:30]

,trade_date,close,return_daily,end_date,Vol,UpperBound,LowerBound,UpperTime,LowerTime,FirstTime
20,2000-03-14,3.25,-0.0031,2000-04-11,0.022144,0.044403,-0.044173,NaT,NaT,2000-04-11
21,2000-03-15,3.21,-0.0123,2000-04-12,0.021379,0.041673,-0.043843,2000-03-23,NaT,2000-03-23
22,2000-03-16,3.13,-0.0249,2000-04-13,0.021564,0.039744,-0.046513,NaT,NaT,2000-04-13
23,2000-03-17,3.18,0.0160,2000-04-14,0.021322,0.041129,-0.044159,NaT,NaT,2000-04-14
24,2000-03-20,3.20,0.0063,2000-04-17,0.020406,0.040049,-0.041574,NaT,NaT,2000-04-17
25,2000-03-21,3.20,0.0000,2000-04-18,0.019400,0.038110,-0.039489,NaT,NaT,2000-04-18
26,2000-03-22,3.24,0.0125,2000-04-19,0.018870,0.038318,-0.037161,NaT,NaT,2000-04-19
27,2000-03-23,3.26,0.0062,2000-04-20,0.018020,0.037159,-0.034923,NaT,NaT,2000-04-20
28,2000-03-24,3.26,0.0000,2000-04-21,0.017137,0.035284,-0.033263,NaT,2000-04-04,2000-04-04
29,2000-03-27,3.31,0.0153,2000-04-24,0.016856,0.036094,-0.031332,NaT,NaT,2000-04-24


In [6]:
# 为了sample weight 加入firsttime
import numpy as np


def get_label(events, df):
    result = events[["trade_date", "FirstTime"]].copy(deep=True)
    price_start = events["close"]
    price_end = pd.merge(df, events, left_on="trade_date",
                         right_on="FirstTime", how='right')["close_x"]
    result["return"] = price_end/price_start-1
    result.dropna(inplace=True)
    result["label"] = np.sign(result["return"])
    # maybe can label 0
    result.loc[events["FirstTime"] == events["end_date"], "label"] = 0

    return result


result = get_label(events, df1)
result.to_csv("../datasets/label_train.csv", index=False)
result.head()

,trade_date,FirstTime,return,label
0,2000-02-15,2000-03-14,0.030488,0.0
1,2000-02-16,2000-03-15,0.049689,0.0
2,2000-02-17,2000-03-09,0.062893,1.0
3,2000-02-18,2000-03-17,0.069620,0.0
4,2000-02-21,2000-03-20,-0.015152,0.0


In [7]:
sr_label_counts = result["label"].value_counts()
df_label = pd.DataFrame(sr_label_counts)
df_label.columns = ["label_nums"]
df_label

,label_nums
-1.0,1632
1.0,1518
0.0,351
